In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import polars
from polars import DataFrame, Series, col
from transformers import AutoModelForSequenceClassification
from label_legends.deberta import load_dataset, load_deberta, load_dataset
from label_legends.preprocess import holdout, load_data, load_test, transform
from label_legends.female import predict_female
from label_legends.swears_negative import predict_swear, predict_negative_sentiment
from label_legends.result import calculate_scores
from label_legends.util import ROOT

/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Use Deberta to create initial predictions:

In [3]:
deberta = load_deberta()

/home/lukas/.local/share/uv/python/cpython-3.12.5-linux-x86_64-gnu/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

  self.pid = os.fork()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/home/lukas/.local/share/uv/python/cpython-3.12.5-linux-x86_64-gnu/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

  self.pid = os.fork()


/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [4]:
train_out = deberta.train()
train_out.metrics

/home/lukas/Programming/uni/nlp-ie-label-legends/src/label_legends/deberta.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss


/home/lukas/Programming/uni/nlp-ie-label-legends/src/label_legends/deberta.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


/home/lukas/Programming/uni/nlp-ie-label-legends/src/label_legends/deberta.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


/home/lukas/Programming/uni/nlp-ie-label-legends/src/label_legends/deberta.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


/home/lukas/Programming/uni/nlp-ie-label-legends/src/label_legends/deberta.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


/home/lukas/Programming/uni/nlp-ie-label-legends/src/label_legends/deberta.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


🏃 View run ./results at: https://mlflow.mahluke.page/#/experiments/0/runs/3ac4da83be204dc79696e67af5d6f2f6
🧪 View experiment at: https://mlflow.mahluke.page/#/experiments/0


{'train_runtime': 3821.6682,
 'train_samples_per_second': 38.465,
 'train_steps_per_second': 2.405,
 'total_flos': 7025499503316000.0,
 'train_loss': 0.2741721511276295,
 'epoch': 5.0}

In [5]:
test = load_test().collect()
test_transformed = transform(test)
deberta_prediction = deberta.predict(load_dataset(test_transformed["text"].to_list(), test_transformed["label"].to_list()))
deberta_prediction

/home/lukas/Programming/uni/nlp-ie-label-legends/src/label_legends/deberta.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


PredictionOutput(predictions=array([[-0.82901084,  0.8538155 ],
       [ 1.5196092 , -0.93007046],
       [ 1.5196092 , -0.93007046],
       ...,
       [ 0.28441924, -0.00646599],
       [ 0.28441924, -0.00646599],
       [ 0.28441924, -0.00646599]], dtype=float32), label_ids=array([1, 0, 0, ..., 1, 0, 1]), metrics={'test_loss': 0.3585379719734192, 'test_precision': 0.7245005257623555, 'test_recall': 0.6635634028892455, 'test_fscore': 0.6926943699731903, 'test_accuracy': 0.8471666666666666, 'test_tp': 2067, 'test_tn': 8099, 'test_fp': 786, 'test_fn': 1048, 'test_runtime': 77.9195, 'test_samples_per_second': 154.005, 'test_steps_per_second': 9.625})

In [6]:
predictions = DataFrame({"id": test["id"], "label": deberta_prediction.label_ids, "deberta": np.argmax(deberta_prediction.predictions, axis=1)})
predictions.head()

id,label,deberta
i64,i64,i64
100,1,1
10005,0,0
10006,0,0
10007,0,0
10008,0,0


In [7]:
scores_deberta = calculate_scores(predictions["label"], predictions["deberta"])
scores_deberta

precision:	0.7245
recall:		0.6636
fscore:		0.6927
accuracy:	0.8472
tn: 8099	 fp: 786
fn: 1048	 tp: 2067

# Post-Process negative predictions

## Predict if sentence is referencing a female

In [8]:
def assign_type(prediction: int, label: str):
    if prediction == 0:
        if label == "not sexist":
            return "tn"
        return "fn"
    if label == "sexist":
        return "tp"
    return "fp"


predictions = predictions.join(test, on="id").with_columns(polars.struct(["deberta", "label_sexist"]).map_elements(lambda x: assign_type(x['deberta'],x['label_sexist']), return_dtype=polars.String).alias("type")).select(["id", "type", "label", "deberta"])

predictions.head()

id,type,label,deberta
i64,str,i64,i64
100,"""tp""",1,1
10005,"""tn""",0,0
10006,"""tn""",0,0
10007,"""tn""",0,0
10008,"""tn""",0,0


Add the predictions whether a text is about a female or not:

In [9]:
def alt_on_neg_pred(prediction: int, alternative: int):
    """Use the alternative if prediction is negative"""
    if prediction == 1:
        return 1
    return alternative


predictions = predictions.join(predict_female(test), on="id").with_columns(
    polars.struct(["female", "deberta"]).map_elements(lambda x: alt_on_neg_pred(x["deberta"], x["female"])).alias("pred_female"))

/tmp/ipykernel_281774/2309025452.py:8: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  predictions = predictions.join(predict_female(test), on="id").with_columns(


We see that the prediction of most false positives is reconsidered, as those contain some female phrase. However, many of true negatives are also reconsidered, leading to a potentially worse precision. As we are focusing on increasing the recall, this is a trade-off we have to make.

In [10]:
predictions.group_by("female", "type").len().with_columns((polars.col("len") / polars.sum("len").over("type")).alias("perc")).filter(col("female")==1).sort("len").rename({"len": "samples"})

female,type,samples,perc
i32,str,u32,f64
1,"""fp""",685,0.871501
1,"""fn""",934,0.891221
1,"""tp""",1766,0.854378
1,"""tn""",5453,0.673293


In [11]:
predictions.head()

id,type,label,deberta,female,pred_female
i64,str,i64,i64,i32,i64
100,"""tp""",1,1,1,1
10005,"""tn""",0,0,1,1
10006,"""tn""",0,0,1,1
10007,"""tn""",0,0,1,1
10008,"""tn""",0,0,1,1


If we would only use 'female' without some additional measurements, we would increase our recall considerably to over 95%, which comes at the cost of precision and f-score, which are drastically decreased. In order to not worsen the model that strongly overall, we add additional measurements to decide when to change a negative prediction by Deberta to a positive one.

In [12]:
scores_female = calculate_scores(predictions["label"], predictions["pred_female"])
scores_female

precision:	0.3248
recall:		0.9634
fscore:		0.4858
accuracy:	0.4706
tn: 2646	 fp: 6239
fn: 114	 tp: 3001

## Sentiment analysis
Predict if text has a negative connotation. If true, and also targeted towards a female, predict the sample to be sexist.




In [13]:
predictions = predictions.join(predict_negative_sentiment(test), on="id").with_columns(polars.struct(["negative", "deberta"]).map_elements(lambda x: alt_on_neg_pred(x["deberta"], x["negative"])).alias("pred_negative"))
predictions.head()

/tmp/ipykernel_281774/120385903.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  predictions = predictions.join(predict_negative_sentiment(test), on="id").with_columns(polars.struct(["negative", "deberta"]).map_elements(lambda x: alt_on_neg_pred(x["deberta"], x["negative"])).alias("pred_negative"))


id,type,label,deberta,female,pred_female,negative,pred_negative
i64,str,i64,i64,i32,i64,i32,i64
100,"""tp""",1,1,1,1,0,1
10005,"""tn""",0,0,1,1,1,1
10006,"""tn""",0,0,1,1,1,1
10007,"""tn""",0,0,1,1,1,1
10008,"""tn""",0,0,1,1,1,1


In [14]:
predictions.group_by("negative", "type", "female").len().with_columns((polars.col("len") / polars.sum("len").over("type")).alias("perc")).filter(col("type").is_in(["fn", "tn"])).sort("len").rename({"len": "samples"}).sort("type")

negative,type,female,samples,perc
i32,str,i32,u32,f64
1,"""fn""",0,50,0.04771
0,"""fn""",0,64,0.061069
0,"""fn""",1,445,0.424618
1,"""fn""",1,489,0.466603
0,"""tn""",0,1196,0.147673
1,"""tn""",0,1450,0.179034
1,"""tn""",1,2595,0.32041
0,"""tn""",1,2858,0.352883


In [15]:
scores_negative = calculate_scores(predictions["label"], predictions["pred_negative"])
scores_negative

precision:	0.3504
recall:		0.8366
fscore:		0.4939
accuracy:	0.5550
tn: 4054	 fp: 4831
fn: 509	 tp: 2606

### Combine female & negative

In [16]:
predictions = predictions.with_columns(polars.struct(["pred_negative", "pred_female"]).map_elements(lambda x: x["pred_negative"] and x["pred_female"]).alias("pred_female_negative"))

/tmp/ipykernel_281774/730595171.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  predictions = predictions.with_columns(polars.struct(["pred_negative", "pred_female"]).map_elements(lambda x: x["pred_negative"] and x["pred_female"]).alias("pred_female_negative"))


In [17]:
scores_female_negative = calculate_scores(predictions["label"], predictions["pred_female_negative"])
scores_female_negative

precision:	0.4305
recall:		0.8205
fscore:		0.5647
accuracy:	0.6717
tn: 5504	 fp: 3381
fn: 559	 tp: 2556

## Bad words
Check if text contains a 'bad word', if so, predict the sample to be sexist.

In [18]:
predictions = predictions.join(predict_swear(test), on="id").with_columns(
    polars.struct(["swear", "deberta"]).map_elements(lambda x: alt_on_neg_pred(x["deberta"], x["swear"])).alias("pred_swear"))

/tmp/ipykernel_281774/1739650169.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  predictions = predictions.join(predict_swear(test), on="id").with_columns(


In [19]:
predictions.group_by("negative", "swear", "type").len().with_columns((polars.col("len") / polars.sum("len").over("type")).alias("perc")).filter(col("type").is_in(["fn", "tn"])).sort("len").rename({"len": "samples"})

negative,swear,type,samples,perc
i32,i32,str,u32,f64
0,1,"""fn""",152,0.145038
1,0,"""fn""",246,0.234733
1,1,"""fn""",293,0.27958
0,0,"""fn""",357,0.340649
0,1,"""tn""",1003,0.123842
1,1,"""tn""",1999,0.246821
1,0,"""tn""",2046,0.252624
0,0,"""tn""",3051,0.376713


In [20]:
scores_swear = calculate_scores(predictions["label"], predictions["pred_swear"])
scores_swear

precision:	0.3987
recall:		0.8064
fscore:		0.5336
accuracy:	0.6341
tn: 5097	 fp: 3788
fn: 603	 tp: 2512

## Ensemble
Check if text is about a female AND if it contains swear words, or if text is about a female AND it is negative. If so, predict the sample to be sexist.


In [21]:
predictions = predictions.with_columns(
    polars.struct(["pred_female_negative", "pred_swear"]).map_elements(lambda x: x["pred_female_negative"] and x["pred_swear"]).alias("pred_ensemble"))

/tmp/ipykernel_281774/1595796645.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  predictions = predictions.with_columns(


In [22]:
scores_ensemble = calculate_scores(predictions["label"], predictions["pred_ensemble"])
scores_ensemble

precision:	0.5364
recall:		0.7470
fscore:		0.6244
accuracy:	0.7668
tn: 6874	 fp: 2011
fn: 788	 tp: 2327

# Store results

In [23]:
scores_df = DataFrame([scores.asdict() for scores in [scores_deberta, scores_female, scores_negative, scores_swear, scores_female_negative, scores_ensemble]]).with_columns_seq(Series(name="model", values=["deberta", "female", "negative", "swear", "female_negative", "ensemble"])).select("model", "precision", "recall", "fscore", "accuracy", "tp", "tn", "fp","fn")
scores_df.head()

model,precision,recall,fscore,accuracy,tp,tn,fp,fn
str,f64,f64,f64,f64,i64,i64,i64,i64
"""deberta""",0.724501,0.663563,0.692694,0.847167,2067,8099,786,1048
"""female""",0.324784,0.963403,0.485795,0.470583,3001,2646,6239,114
"""negative""",0.35041,0.836597,0.493935,0.555,2606,4054,4831,509
"""swear""",0.39873,0.806421,0.533617,0.634083,2512,5097,3788,603
"""female_negative""",0.43052,0.820546,0.564737,0.671667,2556,5504,3381,559


In [24]:
scores_df.write_csv(ROOT / "resource" / "ensemble_scores.csv")

In [25]:
predictions.write_csv(ROOT / "resource" / "ensemble_predictions.csv")